In [1]:
from tweebo_parser import API, ServerError
import numpy as np
import pandas as pd

import sys
import re
import string
import csv
import random
import time
import emoji
import trie
import datetime

import NE_candidate_module as ne
import Mention
# import phase2_Trie_baseline_reintroduction_effectiveness as phase2

# import twokenize

from itertools import groupby
from operator import itemgetter
from collections import Iterable, OrderedDict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from scipy import stats

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


In [2]:
def rreplace(s, old, new, occurrence):
    if s.endswith(old):
        li = s.rsplit(old, occurrence)
        return new.join(li)
    else:
        return s

In [3]:
cachedStopWords = stopwords.words("english")
tempList=["i","and","or","other","another","across","unlike","anytime","were","you","then","still","till","nor","perhaps","otherwise","until","sometimes","sometime","seem","cannot","seems","because","can","like","into","able","unable","either","neither","if","we","it","else","elsewhere","how","not","what","who","when","where","who's","who’s","let","today","tomorrow","tonight","let's","let’s","lets","know","make","oh","via","i","yet","must","mustnt","mustn't","mustn’t","i'll","i’ll","you'll","you’ll","we'll","we’ll","done","doesnt","doesn't","doesn’t","dont","don't","don’t","did","didnt","didn't","didn’t","much","without","could","couldn't","couldn’t","would","wouldn't","wouldn’t","should","shouldn't","souldn’t","shall","isn't","isn’t","hasn't","hasn’t","wasn't","wasn’t","also","let's","let’s","let","well","just","everyone","anyone","noone","none","someone","theres","there's","there’s","everybody","nobody","somebody","anything","else","elsewhere","something","nothing","everything","i'd","i’d","i’m","won't","won’t","i’ve","i've","they're","they’re","we’re","we're","we'll","we’ll","we’ve","we've","they’ve","they've","they’d","they'd","they’ll","they'll","again","you're","you’re","you've","you’ve","thats","that's",'that’s','here’s',"here's","what's","what’s","i’m","i'm","a","so","except","arn't","aren't","arent","this","when","it","it’s","it's","he's","she's","she'd","he'd","he'll","she'll","she’ll","many","can't","cant","can’t","even","yes","no","these","here","there","to","maybe","<hashtag>","<hashtag>.","ever","every","never","there's","there’s","whenever","wherever","however","whatever","always","although"]
for item in tempList:
    if item not in cachedStopWords:
        cachedStopWords.append(item)
cachedStopWords.remove("don")
cachedStopWords.remove("your")
cachedStopWords.remove("up")
cachedTitles = ["mr.","mr","mrs.","mrs","miss","ms","sen.","dr","dr.","prof.","president","congressman"]
prep_list=["in","at","of","on","v."] #includes common conjunction as well
article_list=["a","an","the"]
conjoiner=["de"]
day_list=["sunday","monday","tuesday","wednesday","thursday","friday","saturday","mon","tues","wed","thurs","fri","sat","sun"]
month_list=["january","february","march","april","may","june","july","august","september","october","november","december","jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
chat_word_list=["nope","gee","hmm","bye","vs","ouch","omw","qt","dj","dm","congrat","haueheuaeh","ahushaush","jr","please","retweet","2mrw","2moro","4get","ooh","reppin","idk","oops","yup","stfu","uhh","2b","dear","yay","btw","ahhh","b4","ugh","ty","cuz","coz","sorry","yea","asap","ur","bs","rt","lmfao","lfmao","slfmao","u","r","nah","umm","ummm","thank","thanks","congrats","whoa","rofl","ha","ok","okay","hey","hi","huh","ya","yep","yeah","fyi","duh","damn","lol","omg","congratulations","fucking","fuck","f*ck","wtf","wth","aka","wtaf","xoxo","rofl","imo","wow","fck","haha","hehe","hoho"]

string.punctuation=string.punctuation+'…‘’'
#string.punctuation.extend('“','’','”')
#---------------------Existing Lists--------------------

gutenberg_text = ""
for file_id in gutenberg.fileids():
    gutenberg_text += gutenberg.raw(file_id)
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(gutenberg_text)
my_sentence_tokenizer = PunktSentenceTokenizer(trainer.get_params())
my_sentence_tokenizer._params.abbrev_types.add('dr')
my_sentence_tokenizer._params.abbrev_types.add('c.j')
my_sentence_tokenizer._params.abbrev_types.add('u.s')
my_sentence_tokenizer._params.abbrev_types.add('u.s.a')
my_sentence_tokenizer._params.abbrev_types.add('ret.')
my_sentence_tokenizer._params.abbrev_types.add('rep.')



In [4]:
all_stopwords=cachedStopWords+cachedTitles+prep_list+article_list+conjoiner+day_list+month_list+chat_word_list

In [5]:
# Assumes server is running locally at 0.0.0.0:8000
tweebo_api = API()
proper_noun_tag='^'
common_noun_tag='N'
prep_tag='P'


def flatten(mylist, outlist,ignore_types=(str, bytes, int)):
    
    if mylist !=[]:
        for item in mylist:
            #print not isinstance(item, ne.NE_candidate)
            if isinstance(item, list) and not isinstance(item, ignore_types):
                flatten(item, outlist)
            else:
#                 if isinstance(item,ne.NE_candidate):
#                     item.phraseText=item.phraseText.strip(' \t\n\r')
#                     item.reset_length()
#                 else:
                if type(item)!= int:
                    item=item.strip(' \t\n\r')
                outlist.append(item)
    return outlist
    
def splitSentence(tweetText):
#     print(tweetText)
    tweetSentences=list(filter (lambda sentence: len(sentence)>1, tweetText.split('\n')))
    # tweetSentenceList_inter=self.flatten(list(map(lambda sentText: sent_tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList_inter= flatten(list(map(lambda sentText: my_sentence_tokenizer.tokenize(sentText.lstrip().rstrip()),tweetSentences)),[])
    tweetSentenceList=list(filter (lambda sentence: len(sentence)>1, tweetSentenceList_inter))
    return tweetSentenceList

In [6]:
def getWords(sentence):
    tempList=[]
    tempWordList=sentence.split()
    p_dots= re.compile(r'[.]{2,}')
    #print(tempWordList)
    for word in tempWordList:
        temp=[]

        if "(" in word:
            temp=list(filter(lambda elem: elem!='',word.split("(")))
            if(temp):
                temp=list(map(lambda elem: '('+elem, temp))
        elif ")" in word:
            temp=list(filter(lambda elem: elem!='',word.split(")")))
            if(temp):
                temp=list(map(lambda elem: elem+')', temp))
            # temp.append(temp1[-1])
        elif (("-" in word)&(not word.endswith("-"))):
            temp1=list(filter(lambda elem: elem!='',word.split("-")))
            if(temp1):
                temp=list(map(lambda elem: elem+'-', temp1[:-1]))
            temp.append(temp1[-1])
        elif (("?" in word)&(not word.endswith("?"))):
            temp1=list(filter(lambda elem: elem!='',word.split("?")))
            if(temp1):
                temp=list(map(lambda elem: elem+'?', temp1[:-1]))
            temp.append(temp1[-1])
        elif ((":" in word)&(not word.endswith(":"))):
            temp1=list(filter(lambda elem: elem!='',word.split(":")))
            if(temp1):
                temp=list(map(lambda elem: elem+':', temp1[:-1]))
            temp.append(temp1[-1])
        elif (("," in word)&(not word.endswith(","))):
            #temp=list(filter(lambda elem: elem!='',word.split(",")))
            temp1=list(filter(lambda elem: elem!='',word.split(",")))
            if(temp1):
                temp=list(map(lambda elem: elem+',', temp1[:-1]))
            temp.append(temp1[-1])
        elif (("/" in word)&(not word.endswith("/"))):
            temp1=list(filter(lambda elem: elem!='',word.split("/")))
            if(temp1):
                temp=list(map(lambda elem: elem+'/', temp1[:-1]))
            temp.append(temp1[-1])
        elif (list(p_dots.finditer(word))):
            matched_spans= list(p_dots.finditer(word)) 
            temp=[]
            next_string_start=0
            for matched_span in matched_spans:
                matched_start=matched_span.span()[0]
                this_excerpt=word[next_string_start:matched_start]
                if(this_excerpt):
                    temp.append(this_excerpt)
                next_string_start=matched_span.span()[1]
            if(next_string_start<len(word)):
                last_excerpt=word[next_string_start:]
                if(last_excerpt):
                    temp.append(last_excerpt)
        elif "…" in word:
            temp=list(filter(lambda elem: elem!='',word.split("…")))
            if(temp):
                if(word.endswith("…")):
                    temp=list(map(lambda elem: elem+'…', temp))
                else:
                    temp=list(map(lambda elem: elem+'…', temp[:-1]))+[temp[-1]]
        else:
            #if word not in string.punctuation:
            temp=[word]
        if(temp):
            tempList.append(temp)
    tweetWordList=flatten(tempList,[])
    return tweetWordList

In [7]:
def getWordsII(sentence):
    tempList=[]
    tempWordList=sentence.split()
    p_dots= re.compile(r'[.]{2,}')
    #print(tempWordList)
    for word in tempWordList:
        if (list(p_dots.finditer(word))):
#             print('==>',word)
            matched_spans= list(p_dots.finditer(word)) 
            temp=[]
            next_string_start=0
            for matched_span in matched_spans:
                matched_start=matched_span.span()[0]
                this_excerpt=word[next_string_start:matched_start]
                if(this_excerpt):
                    temp.append(this_excerpt)
                next_string_start=matched_span.span()[1]
            if(next_string_start<len(word)):
                last_excerpt=word[next_string_start:]
                if(last_excerpt):
                    temp.append(last_excerpt)
#             print(temp)
        elif((word.count('.')==1)&(word.endswith('.'))):
            words=list(filter(lambda elem: elem!='',re.split("(\.)",word)))
            temp=[]
            for token in words:
                if(token!='.'):
                    temp+=list(filter(lambda elem: elem!='',re.split('([^a-zA-Záéíó@#’0-9\'])',token)))
                else:
                    temp.append('.')
        else:
            temp=list(filter(lambda elem: elem!='',re.split('([^a-zA-Záéíó@.#’\'0-9])',word)))
        if(temp):
            tempList.append(temp)
    tweetWordList=flatten(tempList,[])
    return tweetWordList

In [8]:
def normalize(strip_op):
#     strip_op=word
    strip_op=(((strip_op.lstrip(string.punctuation)).rstrip(string.punctuation)).strip()).lower()
    strip_op=(strip_op.lstrip('“‘’”')).rstrip('“‘’”')
    strip_op= rreplace(rreplace(rreplace(strip_op,"'s","",1),"’s","",1),"’s","",1)
#     if strip_op.endswith("'s"):
#         li = strip_op.rsplit("'s", 1)
#         return ''.join(li)
#     elif strip_op.endswith("’s"):
#         li = strip_op.rsplit("’s", 1)
#         return ''.join(li)
#     else:
#         return strip_op
    return strip_op

def split_apostrophe(strip_op):
    if strip_op.endswith("'s"):
#         print('==>',strip_op)
        li = strip_op.rfind("'s")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("’s"):
        li = strip_op.rfind("’s")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("'S"):
#         print('==>',strip_op)
        li = strip_op.rfind("'S")
        return [strip_op[:li],strip_op[li:]]
    elif strip_op.endswith("’S"):
#         print('==>',strip_op)
        li = strip_op.rfind("’S")
        return [strip_op[:li],strip_op[li:]]
    else:
        return [strip_op]
#     return strip_op
    
def get_encoding_seq(tweet_word_list, mentions):
#     print(tweet_word_list)
#     print(mentions)
    tweet_word_index=0
    encoded_tag_sequence=[]
    while(mentions):
        current_mention=[token.strip() for token in mentions.pop(0).split(' ')]
        while(normalize(current_mention[0])!=normalize(tweet_word_list[tweet_word_index])):
            encoded_tag_sequence.append('O')
            tweet_word_index+=1
        if(normalize(current_mention[0])==normalize(tweet_word_list[tweet_word_index])):
            for token_index, token in enumerate(current_mention):
                if(token_index==0):
                    encoded_tag_sequence.append('B')
                else:
                    encoded_tag_sequence.append('I')
                tweet_word_index+=1
    while(tweet_word_index<len(tweet_word_list)):
        encoded_tag_sequence.append('O')
        tweet_word_index+=1
        
#     print(encoded_tag_sequence)
    untagged=[(tweet_word_list[i],'O') for i in range(len(tweet_word_list))]
    
    return [(tweet_word_list[i],encoded_tag_sequence[i]) for i in range(len(tweet_word_list))], untagged


In [9]:
def get_entities(word_tag_tuples):
    
    mentions=[]
    candidateMention=''
    #emoji.get_emoji_regexp().sub(u'', candidateMention)
    for tup in word_tag_tuples:
        candidate=tup[0]
        tag=tup[1]
        if(tag=='O'):
            if(candidateMention):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append(mention_to_add)
            candidateMention=''
        else:
            if (tag=='B'):
                if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
                    mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
                    if mention_to_add.endswith("'s"):
                        li = mention_to_add.rsplit("'s", 1)
                        mention_to_add=''.join(li)
                    elif mention_to_add.endswith("’s"):
                        li = mention_to_add.rsplit("’s", 1)
                        mention_to_add=''.join(li)
                    else:
                        mention_to_add=mention_to_add
                    if(mention_to_add!=''):
                        mentions.append(mention_to_add)
                candidateMention=candidate
            else:
                candidateMention+=" "+candidate
        # if (tag=='B'):
        #     if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))):
        #         mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
        #         if(mention_to_add):
        #             mentions.append(mention_to_add)
        #     candidateMention=candidate
        # else:
        #     candidateMention+=" "+candidate
    if(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).strip()):
        if((not candidateMention.strip().startswith('#'))&(not candidateMention.strip().startswith('@'))&(not candidateMention.strip().startswith('https:'))):
            mention_to_add=emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip()
            if(mention_to_add!=''):
                mentions.append(mention_to_add)
        # mentions.append(emoji.get_emoji_regexp().sub(u'', candidateMention).strip(string.punctuation).lower().strip())
    # print('extracted mentions:', mentions)
    return mentions

In [10]:
def get_F1(annotated_mention_list,output_mentions_list):

    # print(tweetID,annotated_mention_list,output_mentions_list)
    unrecovered_annotated_mention_list=[]
    tp_counter_inner=0
    fp_counter_inner=0
    fn_counter_inner=0
    all_postitive_counter_inner=len(output_mentions_list)
    while(annotated_mention_list):
        if(len(output_mentions_list)):
            annotated_candidate= annotated_mention_list.pop()
            if(annotated_candidate in output_mentions_list):
                output_mentions_list.pop(output_mentions_list.index(annotated_candidate))
                tp_counter_inner+=1
            else:
                unrecovered_annotated_mention_list.append(annotated_candidate)
        else:
            unrecovered_annotated_mention_list.extend(annotated_mention_list)
            break
    # unrecovered_annotated_mention_list_outer.extend(unrecovered_annotated_mention_list)
    fn_counter_inner=len(unrecovered_annotated_mention_list)
    fp_counter_inner=all_postitive_counter_inner- tp_counter_inner

    print(tp_counter_inner,fp_counter_inner,fn_counter_inner)
    
    precision=(tp_counter_inner)/(tp_counter_inner+fp_counter_inner)
    recall=(tp_counter_inner)/(tp_counter_inner+fn_counter_inner)
    f_measure=2*(precision*recall)/(precision+recall)
            
    print('precision: ',precision)
    print('recall: ',recall)
    print('f_measure: ',f_measure)

In [11]:
filenames=['ripcity','pikapika','roevwade','billnye','billdeblasio']
fileMentionLists=[]
fileSentenceLists=[]
filePOSTaggedSentenceLists=[]
fileBioSentencesLists=[]
fileUntaggedSentencesLists=[]
fileSentenceLevelMentionsLists=[]

for filename in filenames:

    tweets_unpartitoned=pd.read_csv("/Users/satadisha/Documents/GitHub/"+filename+".csv",sep =',',keep_default_na=False)
    print(len(tweets_unpartitoned))

    df_holder=[]
    batch_number=0
    # tweetList=[]
    # sentenceList=[]
    sentID=0
    # sentID_to_tweet_ID={}
    tweet_ID_to_sentID={}
    mentionList=[]

    for row in tweets_unpartitoned.itertuples():

        index=row.Index

        now = datetime.datetime.now()
        hashtags=str(row.HashTags)

        user=str(row.User)
        tweetText=str(row.TweetText)
        annot_raw=""
        stanford_candidates=""
        ritter_candidates = ""
        calai_candidates=""

        ne_List_final=[]
        userMention_List_final=[]
        tweetSentenceList=splitSentence(tweetText)
    #     sentenceList.extend(tweetSentenceList)

        sentIDList=[]

        for sentence in tweetSentenceList:
            sentIDList.append(sentID)
            sentID+=1
    #         sentID_to_tweet_ID[sentID]=int(index)
    #         sentID+=1

        tweet_ID_to_sentID[index]=sentIDList

        mentions=[]
        for sentence_level in str(row.mentions_other).split(';'):
            if(sentence_level):
                for mention in sentence_level.split(','):
                    if(mention):
                        mentions.append(mention.strip())
        mentionList.append(mentions)
    
    fileMentionLists.append(mentionList)

# # do sentence wise parsing here
# try:
#     result_conll = tweebo_api.parse_conll(sentenceList)
# except:
#     print(f'{e}\n{e.message}')
# print('parse done!')

# sentences=[]
# for sentId, sentence in enumerate(sentenceList):
#     result=result_conll[sentId]
#     result_lines=result.split('\n')
#     sent_tokens=[]
#     for line in result_lines:
#         tabs=line.split('\t')
#         sent_tokens.append(tabs[0])
#     sentences.append(sent_tokens)


# for filename in filenames:
    #or read from parsed files and tally len of sentences
    f = open("/Users/satadisha/Documents/GitHub/BIO_annotations/twokenized_BIO_gaguilar/"+filename+"_BIOannotated_twokenized_POSTAG.txt", "r")
    file_text=f.read()
    print('for tally:',len(file_text.split('\n\n')))
    sentenceList=[]
    pos_tagged_sentences=list(filter (lambda elem: elem!='', file_text.split('\n\n'))) #conll
    print(len(pos_tagged_sentences))
    for pos_tagged_sentence in pos_tagged_sentences:
        sentence_tokens=[]
        annotation=[]
        lines=pos_tagged_sentence.split('\n')
        for line in lines:
            if(line):
                tabs=line.split('\t')
    # 	        if not((tabs[0]=='')&(tabs[1]=='')):
                sentence_tokens.append(tabs[0])
        sentenceList.append(sentence_tokens)
    fileSentenceLists.append(sentenceList)
    filePOSTaggedSentenceLists.append(pos_tagged_sentences)
    
    print(len(pos_tagged_sentences),len(sentenceList))

    # extract sentences for each tweet
    sentID=0
    write_text=''
    write_text_untagged=''
    sentence_level_mentions=[]

    for tweetID in tweet_ID_to_sentID.keys():
        sentIDs=tweet_ID_to_sentID[tweetID]
        mentions=mentionList[tweetID]
        all_tweet_tokens=[]
        sentence_limits=[]
#         print(tweetID)
        start=0
        for sentID in sentIDs:
            all_tweet_tokens+=sentenceList[sentID]
            sentence_limits.append((start,start+len(sentenceList[sentID])))
            start+=len(sentenceList[sentID])
        tweet_word_encoding_list, untagged= get_encoding_seq(all_tweet_tokens, mentions)
    #     print(tweet_word_encoding_list)
        for tup in sentence_limits:
            sentence_word_tag_tuples=[]
            for index in range(tup[0],tup[1]):
                write_text+=tweet_word_encoding_list[index][0]+'\t'+tweet_word_encoding_list[index][1]+'\n'
                sentence_word_tag_tuples.append((tweet_word_encoding_list[index][0],tweet_word_encoding_list[index][1]))
                write_text_untagged+=untagged[index][0]+'\t'+untagged[index][1]+'\n'
            write_text+='\n'
            write_text_untagged+='\n'

            sentence_level_mentions.append(get_entities(sentence_word_tag_tuples))

    bio_sentences=list(filter (lambda elem: elem!='', write_text.split('\n\n')))
    untagged_sentences=list(filter (lambda elem: elem!='', write_text_untagged.split('\n\n')))
    print(len(bio_sentences),len(untagged_sentences),len(sentence_level_mentions))
    
    fileBioSentencesLists.append(bio_sentences)
    fileUntaggedSentencesLists.append(untagged_sentences)
    fileSentenceLevelMentionsLists.append(sentence_level_mentions)

1007
for tally: 2620
2619
2619 2619
2619 2619 2619
2004
for tally: 4979
4978
4978 4978
4978 4978 4978
1009
for tally: 2527
2526
2526 2526
2526 2526 2526
1004
for tally: 3085
3084
3084 3084
3084 3084 3084
1054
for tally: 3755
3754
3754 3754
3754 3754 3754


In [12]:
# print(bio_sentences)
print(len(fileBioSentencesLists))
print(len(fileSentenceLevelMentionsLists))
print(len(fileUntaggedSentencesLists))
print(len(filePOSTaggedSentenceLists))
# different merge iterations

5
5
5


In [14]:
split1_train=[0,1,2,3]
split1_test=4

train_total=0

# write_train_pos = open("/Users/satadisha/Documents/GitHub/3A/split1.train.pos", "w")
# write_train_bio = open("/Users/satadisha/Documents/GitHub/3A/split1.train.bio", "w")

# write_test_pos = open("/Users/satadisha/Documents/GitHub/3A/split1.test.pos", "w")
# write_test_untagged = open("/Users/satadisha/Documents/GitHub/3A/split1.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

for index in split1_train:
    pos_tagged_sentences=filePOSTaggedSentenceLists[index]
    train_total+=len(pos_tagged_sentences)
    for sentence in pos_tagged_sentences:
        train_text_pos+=sentence+'\n\n'
    
    bio_tagged_sentences=fileBioSentencesLists[index]
    for sentence in bio_tagged_sentences:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

# write_train_pos.write(train_text_pos)
# write_train_pos.close()

# write_train_bio.write(train_text_BIO)
# write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=filePOSTaggedSentenceLists[split1_test]
test_untagged_fold=fileUntaggedSentencesLists[split1_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
# write_test_pos.write(test_text_pos)
# write_test_pos.close()

# write_test_untagged.write(test_text_untagged)
# write_test_untagged.close()

train fold: 13207
3754 3754


In [16]:
## evaluation
mention_fold=fileSentenceLevelMentionsLists[split1_test]
all_mentions=[]
for mentions in mention_fold:
    all_mentions+=mentions
print('all tagged mentions:',len(all_mentions))

f = open('/Users/satadisha/Documents/GitHub/3A/split1_output.txt')
output_text=f.read()
all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
print('all output mentions:',len(all_outputs))

get_F1(all_mentions,all_outputs)

all tagged mentions: 1193
all output mentions: 1025
933 92 260
precision:  0.9102439024390244
recall:  0.7820620284995808
f_measure:  0.8412984670874661


In [18]:
split2_train=[0,1,2,4]
split2_test=3

train_total=0

write_train_pos = open("/Users/satadisha/Documents/GitHub/3A/split2.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/3A/split2.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/3A/split2.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/3A/split2.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

for index in split2_train:
    pos_tagged_sentences=filePOSTaggedSentenceLists[index]
    train_total+=len(pos_tagged_sentences)
    for sentence in pos_tagged_sentences:
        train_text_pos+=sentence+'\n\n'
    
    bio_tagged_sentences=fileBioSentencesLists[index]
    for sentence in bio_tagged_sentences:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=filePOSTaggedSentenceLists[split2_test]
test_untagged_fold=fileUntaggedSentencesLists[split2_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

train fold: 13877
3084 3084


In [19]:
## evaluation
mention_fold=fileSentenceLevelMentionsLists[split2_test]
all_mentions=[]
for mentions in mention_fold:
    all_mentions+=mentions
print('all tagged mentions:',len(all_mentions))

f = open('/Users/satadisha/Documents/GitHub/3A/split2_output.txt')
output_text=f.read()
all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
print('all output mentions:',len(all_outputs))

get_F1(all_mentions,all_outputs)

all tagged mentions: 794
all output mentions: 588
431 157 363
precision:  0.7329931972789115
recall:  0.5428211586901763
f_measure:  0.6237337192474675


In [20]:
split3_train=[0,1,3,4]
split3_test=2

train_total=0

write_train_pos = open("/Users/satadisha/Documents/GitHub/3A/split3.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/3A/split3.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/3A/split3.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/3A/split3.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

for index in split3_train:
    pos_tagged_sentences=filePOSTaggedSentenceLists[index]
    train_total+=len(pos_tagged_sentences)
    for sentence in pos_tagged_sentences:
        train_text_pos+=sentence+'\n\n'
    
    bio_tagged_sentences=fileBioSentencesLists[index]
    for sentence in bio_tagged_sentences:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=filePOSTaggedSentenceLists[split3_test]
test_untagged_fold=fileUntaggedSentencesLists[split3_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

train fold: 14435
2526 2526


In [21]:
## evaluation
mention_fold=fileSentenceLevelMentionsLists[split3_test]
all_mentions=[]
for mentions in mention_fold:
    all_mentions+=mentions
print('all tagged mentions:',len(all_mentions))

f = open('/Users/satadisha/Documents/GitHub/3A/split3_output.txt')
output_text=f.read()
all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
print('all output mentions:',len(all_outputs))

get_F1(all_mentions,all_outputs)

all tagged mentions: 1823
all output mentions: 1363
1301 62 522
precision:  0.954512105649303
recall:  0.7136588041689522
f_measure:  0.8166980539861894


In [22]:
split4_train=[0,2,3,4]
split4_test=1

train_total=0

write_train_pos = open("/Users/satadisha/Documents/GitHub/3A/split4.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/3A/split4.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/3A/split4.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/3A/split4.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

for index in split4_train:
    pos_tagged_sentences=filePOSTaggedSentenceLists[index]
    train_total+=len(pos_tagged_sentences)
    for sentence in pos_tagged_sentences:
        train_text_pos+=sentence+'\n\n'
    
    bio_tagged_sentences=fileBioSentencesLists[index]
    for sentence in bio_tagged_sentences:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=filePOSTaggedSentenceLists[split4_test]
test_untagged_fold=fileUntaggedSentencesLists[split4_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

train fold: 11983
4978 4978


In [23]:
## evaluation
mention_fold=fileSentenceLevelMentionsLists[split4_test]
all_mentions=[]
for mentions in mention_fold:
    all_mentions+=mentions
print('all tagged mentions:',len(all_mentions))

f = open('/Users/satadisha/Documents/GitHub/3A/split4_output.txt')
output_text=f.read()
all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
print('all output mentions:',len(all_outputs))

get_F1(all_mentions,all_outputs)

all tagged mentions: 1148
all output mentions: 874
770 104 378
precision:  0.8810068649885584
recall:  0.6707317073170732
f_measure:  0.76162215628091


In [24]:
split5_train=[1,2,3,4]
split5_test=0

train_total=0

write_train_pos = open("/Users/satadisha/Documents/GitHub/3A/split5.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/3A/split5.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/3A/split5.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/3A/split5.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

for index in split5_train:
    pos_tagged_sentences=filePOSTaggedSentenceLists[index]
    train_total+=len(pos_tagged_sentences)
    for sentence in pos_tagged_sentences:
        train_text_pos+=sentence+'\n\n'
    
    bio_tagged_sentences=fileBioSentencesLists[index]
    for sentence in bio_tagged_sentences:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=filePOSTaggedSentenceLists[split5_test]
test_untagged_fold=fileUntaggedSentencesLists[split5_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

train fold: 14342
2619 2619


In [25]:
## evaluation
mention_fold=fileSentenceLevelMentionsLists[split5_test]
all_mentions=[]
for mentions in mention_fold:
    all_mentions+=mentions
print('all tagged mentions:',len(all_mentions))

f = open('/Users/satadisha/Documents/GitHub/3A/split5_output.txt')
output_text=f.read()
all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
print('all output mentions:',len(all_outputs))

get_F1(all_mentions,all_outputs)

all tagged mentions: 1105
all output mentions: 765
693 72 412
precision:  0.9058823529411765
recall:  0.6271493212669683
f_measure:  0.7411764705882353


## Experiment 3B setup beyond this point

In [25]:
length = int(len(bio_sentences)/5) #length of each fold
pos_tagged_folds = []
bio_tagged_folds= []
untagged_folds= []
mention_folds= []

for i in range(4):
    pos_tagged_folds += [pos_tagged_sentences[i*length:(i+1)*length]]
    bio_tagged_folds += [bio_sentences[i*length:(i+1)*length]]
    untagged_folds += [untagged_sentences[i*length:(i+1)*length]]
    mention_folds += [sentence_level_mentions[i*length:(i+1)*length]]
    
pos_tagged_folds += [pos_tagged_sentences[4*length:len(pos_tagged_sentences)]]
bio_tagged_folds += [bio_sentences[4*length:len(bio_sentences)]]
untagged_folds += [untagged_sentences[4*length:len(untagged_sentences)]]
mention_folds += [sentence_level_mentions[4*length:len(sentence_level_mentions)]]

print(len(pos_tagged_folds),len(bio_tagged_folds),len(untagged_folds),len(mention_folds))

5 5 5 5


In [26]:
# print(sentence_level_mentions)

[[], [], [], ['de blasio'], [], [], [], [], [], [], [], [], ['de blasio'], [], ['nyc'], [], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], ['new york'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], [], [], [], ['bill'], [], [], [], [], [], ['de blasio'], [], [], [], ['de blasio'], ['second amendment'], ['democrats'], [], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], ['second amendment'], ['democrats'], [], [], [], [], ['de blasio'], ['ocasio'], ['de blasio'], [], [], [], [], [], [], ['de blasio'], ['deblasio'], [], ['doh', 'aspca'], [], [], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], ['de blasio'], [], [], [], 

In [ ]:
split1_train=[0,1,2,3]
split1_test=4

write_train_pos = open("/Users/satadisha/Documents/GitHub/split1.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/split1.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/split1.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/split1.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

train_total=0
for index in split1_train:
    pos_fold=pos_tagged_folds[index]
    train_total+=len(pos_fold)
    for sentence in pos_fold:
        train_text_pos+=sentence+'\n\n'
    
    bio_fold=bio_tagged_folds[index]
    for sentence in bio_fold:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=pos_tagged_folds[split1_test]
test_untagged_fold=untagged_folds[split1_test]

print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()


# ## evaluation of this fold
# mention_fold=mention_folds[split1_test]
# all_mentions=[]
# for mentions in mention_fold:
#     all_mentions+=mentions
# print('all tagged mentions:',len(all_mentions))

# f = open('/Users/satadisha/Documents/GitHub/split1_output_selfTrain.txt')
# output_text=f.read()
# all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
# print('all output mentions:',len(all_outputs))

# get_F1(all_mentions,all_outputs)

In [31]:
split2_train=[0,1,2,4]
split2_test=3

write_train_pos = open("/Users/satadisha/Documents/GitHub/split2.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/split2.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/split2.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/split2.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

train_total=0
for index in split2_train:
    pos_fold=pos_tagged_folds[index]
    train_total+=len(pos_fold)
    for sentence in pos_fold:
        train_text_pos+=sentence+'\n\n'
    
    bio_fold=bio_tagged_folds[index]
    for sentence in bio_fold:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=pos_tagged_folds[split2_test]
test_untagged_fold=untagged_folds[split2_test]
print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

# ## evaluation of this fold
# mention_fold=mention_folds[split2_test]
# all_mentions=[]
# for mentions in mention_fold:
#     all_mentions+=mentions
# print('all tagged mentions:',len(all_mentions))

# f = open('/Users/satadisha/Documents/GitHub/split2_output_selfTrain.txt')
# output_text=f.read()
# all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
# print('all output mentions:',len(all_outputs))

# get_F1(all_mentions,all_outputs)

train fold: 3004
750 750
all tagged mentions: 249
all output mentions: 241
228 13 21
precision:  0.946058091286307
recall:  0.9156626506024096
f_measure:  0.9306122448979591


In [33]:
split3_train=[0,1,3,4]
split3_test=2

write_train_pos = open("/Users/satadisha/Documents/GitHub/split3.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/split3.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/split3.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/split3.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

train_total=0
for index in split3_train:
    pos_fold=pos_tagged_folds[index]
    train_total+=len(pos_fold)
    for sentence in pos_fold:
        train_text_pos+=sentence+'\n\n'
    
    bio_fold=bio_tagged_folds[index]
    for sentence in bio_fold:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=pos_tagged_folds[split3_test]
test_untagged_fold=untagged_folds[split3_test]
print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

# ## evaluation of this fold
# mention_fold=mention_folds[split3_test]
# all_mentions=[]
# for mentions in mention_fold:
#     all_mentions+=mentions
# print('all tagged mentions:',len(all_mentions))

# f = open('/Users/satadisha/Documents/GitHub/split3_output_selfTrain.txt')
# output_text=f.read()
# all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
# print('all output mentions:',len(all_outputs))

# get_F1(all_mentions,all_outputs)

train fold: 3004
750 750
all tagged mentions: 248
all output mentions: 233
212 21 36
precision:  0.9098712446351931
recall:  0.8548387096774194
f_measure:  0.8814968814968815


In [35]:
split4_train=[0,2,3,4]
split4_test=1

write_train_pos = open("/Users/satadisha/Documents/GitHub/split4.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/split4.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/split4.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/split4.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

train_total=0
for index in split4_train:
    pos_fold=pos_tagged_folds[index]
    train_total+=len(pos_fold)
    for sentence in pos_fold:
        train_text_pos+=sentence+'\n\n'
    
    bio_fold=bio_tagged_folds[index]
    for sentence in bio_fold:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=pos_tagged_folds[split4_test]
test_untagged_fold=untagged_folds[split4_test]
print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

# ## evaluation of this fold
# mention_fold=mention_folds[split4_test]
# all_mentions=[]
# for mentions in mention_fold:
#     all_mentions+=mentions
# print('all tagged mentions:',len(all_mentions))

# f = open('/Users/satadisha/Documents/GitHub/split4_output_selfTrain.txt')
# output_text=f.read()
# all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
# print('all output mentions:',len(all_outputs))

# get_F1(all_mentions,all_outputs)

train fold: 3004
750 750
all tagged mentions: 245
all output mentions: 243
230 13 15
precision:  0.9465020576131687
recall:  0.9387755102040817
f_measure:  0.9426229508196722


In [37]:
split5_train=[1,2,3,4]
split5_test=0

write_train_pos = open("/Users/satadisha/Documents/GitHub/split5.train.pos", "w")
write_train_bio = open("/Users/satadisha/Documents/GitHub/split5.train.bio", "w")

write_test_pos = open("/Users/satadisha/Documents/GitHub/split5.test.pos", "w")
write_test_untagged = open("/Users/satadisha/Documents/GitHub/split5.test.untagged", "w")

train_text_pos=''
train_text_BIO=''

train_total=0
for index in split5_train:
    pos_fold=pos_tagged_folds[index]
    train_total+=len(pos_fold)
    for sentence in pos_fold:
        train_text_pos+=sentence+'\n\n'
    
    bio_fold=bio_tagged_folds[index]
    for sentence in bio_fold:
        train_text_BIO+=sentence+'\n\n'

print('train fold:',train_total)

write_train_pos.write(train_text_pos)
write_train_pos.close()

write_train_bio.write(train_text_BIO)
write_train_bio.close()

test_text_pos=''
test_text_untagged=''

test_pos_fold=pos_tagged_folds[split5_test]
test_untagged_fold=untagged_folds[split5_test]
print(len(test_pos_fold),len(test_untagged_fold))

for sentence in test_pos_fold:
    test_text_pos+=sentence+'\n\n'


for sentence in test_untagged_fold:
    test_text_untagged+=sentence+'\n\n'
    
write_test_pos.write(test_text_pos)
write_test_pos.close()

write_test_untagged.write(test_text_untagged)
write_test_untagged.close()

# ## evaluation of this fold
# mention_fold=mention_folds[split5_test]
# all_mentions=[]
# for mentions in mention_fold:
# #     print(mentions)
#     all_mentions+=mentions
# print('all tagged mentions:',len(all_mentions))

# f = open('/Users/satadisha/Documents/GitHub/split5_output_selfTrain.txt')
# output_text=f.read()
# all_outputs=list(filter (lambda elem: elem!='', output_text.split('\n')))
# print('all output mentions:',len(all_outputs))

# get_F1(all_mentions,all_outputs)

train fold: 3004
750 750
all tagged mentions: 221
all output mentions: 219
211 8 10
precision:  0.9634703196347032
recall:  0.9547511312217195
f_measure:  0.959090909090909
